In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr

# Load the pretrained model and tokenizer
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Track chat history
chat_history_ids = None

def chatbot_response(user_input, history=[]):
    global chat_history_ids

    # Encode the user input and append it to the chat history
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append tokens to chat history or start fresh
    if chat_history_ids is None:
        chat_history_ids = new_input_ids
    else:
        chat_history_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)

    # Generate response
    response_ids = model.generate(chat_history_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[:, chat_history_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

# Create Gradio interface
interface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Say something..."),
    outputs="text",
    title="AI Chatbot",
    description="A simple chatbot using Hugging Face DialoGPT."
)

# Launch application
interface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://c6097191c46a836631.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
